1. 통계적 가설검정
- 1-1. 통계적 가설검정의 기본

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

np.random.seed(1111)
%precision 3
%matplotlib inline

In [3]:
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
# 표본 평균
s_mean = np.mean(sample)
s_mean

128.451

- 귀무가설 : 감자튀김의 평균 무게는 130
- 대립가설 : 모평균은 130보다 작음

In [5]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.681

표본 평균이 128.681보다 작다면 귀무가설을 기각되고, 128.681보다 크면 귀무가설이 채택됨

In [6]:
# 검정 통계량
z = (s_mean -130) / np.sqrt(9/14)
z

-1.932

In [7]:
# 임계값
rv = stats.norm()
rv.isf(0.95)

-1.645

검정 통계량이 임계값 보다 작음 -> 귀무가설은 기각되고 평균은 130보다 작다고 판단

In [8]:
# p값
rv.cdf(z)

0.027

p값은 0.027로 유의수준 0.05보다 작은 값 -> 따라서 귀무가설은 기각됨

- 1-2. 단측검정과 양측검정

- 귀무가설 : 모평균은 130
- 대립가설 : 모평균은 130이 아님

In [9]:
# 검정통계량은 단측 검정때와 같음

z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

In [10]:
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

In [11]:
# 누적밀도 함수의 값을 2배로 설정
rv.cdf(z) * 2

0.053

p 값이 0.05보다 크기 때문에 귀무가설을 기각하지 않음

- 1-3. 가설 검정의 두 가지 오류


- 제 1종 오류 : 귀무가설이 옳을 때, 귀무가설을 기각하는 오류
- 제 2종 오류 : 대립가설이 옳을 때, 귀무가설을 채택하는 오류

In [12]:
# 제 1종 오류의 확률
rv = stats.norm(130,3)

In [13]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    
    if z < c:
        cnt += 1
        
cnt / n_samples

0.053

제 1종 오류를 범한 비율은 0.053으로, 대략 5%의 비율로 130g보다 작다고 잘못 탐지함을 의미한다. 제 1종 오류를 범하는 확률을 위험률이라고 한다.

In [14]:
rv = stats.norm(128,3)

In [16]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0

for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.197

2. 기본적인 가설검정
- 2-1. 정규분포의 모평균에 대한 검정 : 모분산을 알고 있는 경우

In [17]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    
    else:
        print('귀무가설을 기각')
        
    
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1- rv.cdf(z)) * 2
    print(f'p값은 {p:.3f}')

In [18]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p값은 0.053


- 2-2. 정규분포의 모분산에 대한 검정

In [23]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설 채택')
        
    else:
        print('귀무가설 기각')
        
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    
    else:
        p = (1-rv.cdf(y)) * 2
    print(f'p값은 {p:.3f}')

In [24]:
pvar_test(sample, 9)

귀무가설 채택
p값은 0.085


- 2-3. 정규분포의 모평균에 대한 검정 : 모분산을 모르는 경우

In [25]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    
    else:
        print('귀무가설을 기각')
        
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1-rv.cdf(t)) * 2
    
    print(f'p값은 {p:.3f}')

In [26]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


In [27]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.455, 0.169)